In [1]:
# Load all packages necessary for analysis
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import re
import os
import timeit
import filecmp

In [2]:
decks_clean = pd.read_csv('~/Documents/MTG/data/decks_clean.csv')

# decks_clean

## Gather

In [3]:
dir_names = []
curdir = 'C:/Users/muroc/Documents/MTG/html_files'

with os.scandir(curdir) as folder:
    for file in folder:
        if file.is_file():
            dir_names.append(file.name)

In [4]:
decks = []
for i in np.arange(np.count_nonzero(dir_names)):
    with open(str(curdir) + '/' + str(dir_names[i])) as file:
        soup = BeautifulSoup(file, 'lxml')
    cards = soup.find(lambda tag: tag.name=='input' and tag.has_attr('name') and tag['name']=='c')
    player = soup.find()
    decks.append(cards)

In [5]:
deck_urls = pd.read_csv('~/Documents/MTG/data/deck_urls.csv')

In [6]:
decks_dict = {"deck_list": decks}
decks_clean = pd.DataFrame(decks_dict)

In [7]:
decks_clean.to_csv('~/Documents/MTG/data/deck_list.csv', index=False)

## Assess

In [8]:
print(dir_names)

['5647382910_Azorius_Aggro.html', 'Aaron_Barich_4c_Sun.html', 'Aaron_Higgins_Abzan_Precinct_One.html', 'Aaro_Nieminen_Boros_Aggro.html', 'Abram_Goldstein_Bant_Control.html', 'Adamantmtg_Mono_Black.html', 'Adam_Boyd_Izzet_Control.html', 'Adam_Fiffles_Sultai_Sacrifice.html', 'Aiden_Brier_Weenie_White.html', 'ajani_alex_Abzan_Midrange.html', 'Aka_Shunsuke_Mono_Green_Aggro.html', 'AKnightWhoSaysNi_Dimir_Midrange.html', 'Alberto_IbaÃñez_Sanchez_UR_Artifacts.html', 'Albert_Bayrante_Reanimator.html', 'Alber_vcf_Grixis_Control.html', 'Alejandro_Collado_Naya_Feather.html', 'Aleksander_Hudson_Gate_Control.html', 'Alessio_Magris_Bant_Food.html', 'Alexander_David_Romero_Mono_Blue.html', 'Alexander_Demura_UR_Prowess.html', 'Alexander_Hayne_4c_Energy.html', 'Alexander_Hayne_Mono_Black_Aggro.html', 'Alex_Barrera_Sultai_Sacrifice.html', 'AlfredoTorres_Esper_Precinct_One.html', 'AlfredoTorres_Naya_Feather.html', 'AliEnWaRe__Mono_Blue_Aggro.html', 'Ali_Aintrazi_Golgari_Control.html', 'Ali_Eldrazi_Jund_G

In [9]:
deck_urls.head()

,url,name,player
0,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Elementals.html.txt,Robert_Lippmann
1,https://www.mtgtop8.com/https://www.mtgtop8.co...,Omnath.html.txt,Clarence_Dews
2,https://www.mtgtop8.com/https://www.mtgtop8.co...,4colour_Good_Stuff.html.txt,Yohan_Dudognon
3,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Sun.html.txt,Aaron_Barich
4,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Energy.html.txt,rushi_00


In [39]:
decks_clean.head()

,url,name,player,deck_list
0,https://www.mtgtop8.com/event?e=23476&amp;d=36...,4c_Elementals,Robert_Lippmann,"['13 Forest', '4 Llanowar Elves', '3 Greenbelt..."
1,https://www.mtgtop8.com/event?e=23375&amp;d=36...,Omnath,Clarence_Dews,"['10 Forest', '7 Island', '4 Llanowar Elves', ..."
2,https://www.mtgtop8.com/event?e=23384&amp;d=36...,4colour_Good_Stuff,Yohan_Dudognon,"['2 Essence Scatter', '4 Glacial Fortress', '7..."
3,https://www.mtgtop8.com/event?e=22218&amp;d=35...,4c_Sun,Aaron_Barich,"['1 Planar Cleansing', '2 Forest', '2 Island',..."
4,https://www.mtgtop8.com/event?e=18688&amp;d=31...,4c_Energy,rushi_00,"['6 Forest', '4 Swamp', '4 Blood Crypt', '4 Ca..."


In [11]:
decks_clean.tail()

,deck_list
869,"<input name=""c"" type=""hidden"" value=""2 Mountai..."
870,"<input name=""c"" type=""hidden"" value=""19 Swamp|..."
871,"<input name=""c"" type=""hidden"" value=""4 Dragons..."
872,"<input name=""c"" type=""hidden"" value=""1 Shock||..."
873,"<input name=""c"" type=""hidden"" value=""3 Glacial..."


In [12]:
decks_clean[decks_clean.deck_list == '[nan]']

,deck_list


In [13]:
decks_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874 entries, 0 to 873
Data columns (total 1 columns):
deck_list    874 non-null object
dtypes: object(1)
memory usage: 6.9+ KB


In [40]:
decks_clean.deck_list[0]

'[\'13 Forest\', \'4 Llanowar Elves\', \'3 Greenbelt Rampager\', \'2 Aethersphere Harvester\', \'3 Heart of Kiran\', "1 Lifecrafter\'s Bestiary", \'2 Cartouche of Knowledge\', \'3 Rhonas the Indomitable\', \'2 Commit-Memory\', \'4 Steel Leaf Champion\', \'3 Hashep Oasis\', \'4 Hinterland Harbor\', \'2 Blossoming Defense\', "2 Nature\'s Way", \'4 Servant of the Conduit\', \'4 Botanical Sanctum\', \'4 Vine Mare\', \'2 Vivien Reid\', \'3 Negate\', \'4 Ghalta, Primal Hunger\', \'4 Thrashing Brontodon\', \'2 Merfolk Branchwalker\', \'"/>\']'

### Assessment
- [done] data from deck_urls.csv and deck_list.csv are of same object
- [done] .html and .txt tags are irrelevant in name column
- [done] `deck_list` may be type that is irrelevant
- [done] `deck_list` column is one huge object
- [done] irrelevant characters in `deck_list` column
- [done] deck_urls needs a copy to work with
- [done] multiple headers in `url` column because i must have ran the function too often
- missing data at the end of the dataset
- [done] deck_list column is not the right data type
- duplicate observations
- deck_list column is not in the right order because the data was reorganized at some point
- decks are incorrectly named

## Clean

#### Define
- append deck_urls.csv and deck_list.csv together

#### Code

In [14]:
deck_urls['deck_list'] = decks_clean

In [15]:
deck_urls.to_csv('~/Documents/MTG/data/decks_clean.csv', index=False)

#### Test

In [16]:
deck_urls.head()

,url,name,player,deck_list
0,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Elementals.html.txt,Robert_Lippmann,"<input name=""c"" type=""hidden"" value=""13 Forest..."
1,https://www.mtgtop8.com/https://www.mtgtop8.co...,Omnath.html.txt,Clarence_Dews,"<input name=""c"" type=""hidden"" value=""10 Forest..."
2,https://www.mtgtop8.com/https://www.mtgtop8.co...,4colour_Good_Stuff.html.txt,Yohan_Dudognon,"<input name=""c"" type=""hidden"" value=""2 Essence..."
3,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Sun.html.txt,Aaron_Barich,"<input name=""c"" type=""hidden"" value=""1 Planar ..."
4,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Energy.html.txt,rushi_00,"<input name=""c"" type=""hidden"" value=""6 Forest|..."


#### Define
- create a copy of `deck_urls`

#### Code

In [17]:
decks_clean = deck_urls.copy()

#### Test

In [18]:
decks_clean.head()

,url,name,player,deck_list
0,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Elementals.html.txt,Robert_Lippmann,"<input name=""c"" type=""hidden"" value=""13 Forest..."
1,https://www.mtgtop8.com/https://www.mtgtop8.co...,Omnath.html.txt,Clarence_Dews,"<input name=""c"" type=""hidden"" value=""10 Forest..."
2,https://www.mtgtop8.com/https://www.mtgtop8.co...,4colour_Good_Stuff.html.txt,Yohan_Dudognon,"<input name=""c"" type=""hidden"" value=""2 Essence..."
3,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Sun.html.txt,Aaron_Barich,"<input name=""c"" type=""hidden"" value=""1 Planar ..."
4,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Energy.html.txt,rushi_00,"<input name=""c"" type=""hidden"" value=""6 Forest|..."


#### Define
- change type of `deck_list` to str

#### Code

In [19]:
decks_clean.deck_list = decks_clean.deck_list.astype('str')

In [20]:
decks_clean.to_csv('~/Documents/MTG/data/decks_clean.csv', index=False)

#### Test

In [21]:
type(decks_clean.deck_list[0])

str

#### Define
- drop .url and .text from the `name` column

#### Code

In [22]:
decks_clean.name = decks_clean.name.str.replace('.html.txt', '')

In [23]:
decks_clean.to_csv('~/Documents/MTG/data/decks_clean.csv', index=False)

#### Test

In [24]:
decks_clean.head()

,url,name,player,deck_list
0,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Elementals,Robert_Lippmann,"<input name=""c"" type=""hidden"" value=""13 Forest..."
1,https://www.mtgtop8.com/https://www.mtgtop8.co...,Omnath,Clarence_Dews,"<input name=""c"" type=""hidden"" value=""10 Forest..."
2,https://www.mtgtop8.com/https://www.mtgtop8.co...,4colour_Good_Stuff,Yohan_Dudognon,"<input name=""c"" type=""hidden"" value=""2 Essence..."
3,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Sun,Aaron_Barich,"<input name=""c"" type=""hidden"" value=""1 Planar ..."
4,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Energy,rushi_00,"<input name=""c"" type=""hidden"" value=""6 Forest|..."


#### Define
- get rid of all unneccessary characters in `deck_list`

#### Code

In [25]:
decks_clean.deck_list = decks_clean.deck_list.str.replace('<input name="c" type="hidden" value=', '')

In [26]:
decks_clean.deck_list = decks_clean.deck_list.str.strip('"')

In [27]:
decks_clean.to_csv('~/Documents/MTG/data/decks_clean.csv', index=False)

#### Test

In [28]:
decks_clean.head()

,url,name,player,deck_list
0,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Elementals,Robert_Lippmann,13 Forest||4 Llanowar Elves||3 Greenbelt Rampa...
1,https://www.mtgtop8.com/https://www.mtgtop8.co...,Omnath,Clarence_Dews,10 Forest||7 Island||4 Llanowar Elves||4 Breed...
2,https://www.mtgtop8.com/https://www.mtgtop8.co...,4colour_Good_Stuff,Yohan_Dudognon,2 Essence Scatter||4 Glacial Fortress||7 Islan...
3,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Sun,Aaron_Barich,1 Planar Cleansing||2 Forest||2 Island||1 Plai...
4,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Energy,rushi_00,6 Forest||4 Swamp||4 Blood Crypt||4 Cauldron F...


#### Define
- split deck_list into multiple objects

#### Code

In [29]:
decks_clean.deck_list = decks_clean.deck_list.str.split(r'\|\|')

In [30]:
decks_clean.to_csv('~/Documents/MTG/data/decks_clean.csv', index=False)

#### Test

In [31]:
decks_clean.head()

,url,name,player,deck_list
0,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Elementals,Robert_Lippmann,"[13 Forest, 4 Llanowar Elves, 3 Greenbelt Ramp..."
1,https://www.mtgtop8.com/https://www.mtgtop8.co...,Omnath,Clarence_Dews,"[10 Forest, 7 Island, 4 Llanowar Elves, 4 Bree..."
2,https://www.mtgtop8.com/https://www.mtgtop8.co...,4colour_Good_Stuff,Yohan_Dudognon,"[2 Essence Scatter, 4 Glacial Fortress, 7 Isla..."
3,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Sun,Aaron_Barich,"[1 Planar Cleansing, 2 Forest, 2 Island, 1 Pla..."
4,https://www.mtgtop8.com/https://www.mtgtop8.co...,4c_Energy,rushi_00,"[6 Forest, 4 Swamp, 4 Blood Crypt, 4 Cauldron ..."


#### Define
- remove first https from `url` column

#### Code

In [32]:
decks_clean.url = decks_clean.url.str.replace('https://www.mtgtop8.com/', '')
decks_clean.url = 'https://www.mtgtop8.com/' + decks_clean.url

In [33]:
decks_clean.to_csv('~/Documents/MTG/data/decks_clean.csv', index=False)

#### Test

In [34]:
decks_clean.url[0]

'https://www.mtgtop8.com/event?e=23476&amp;d=362227&amp;f=ST'

#### Define
- make `deck_list` column a str

#### Code

In [35]:
decks_clean.deck_list = decks_clean.deck_list.astype('str')

In [36]:
decks_clean.to_csv('~/Documents/MTG/data/decks_clean.csv', index=False)

#### Test

In [37]:
type(decks_clean.deck_list[0])

str

#### Define
- put data points in order in `deck_list

#### Code

In [38]:
decks_clean.deck_list.value_counts()

['nan']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     45
['2 Shock', '10 Forest', '9 Mountain', '4 Bonecrusher Giant', '2 Embercleave', '2 Redcap Melee', '4 Once Upon a Time', '4 Questing Beast', '3 Lovestruck Beast', '2 Kraul